In [16]:
import numpy as np
from numpy import exp,max, sqrt
from FERM import Binomial_Option

# Parameters used throughout the exercise as constants
S0 = 100            # Spot price
r = 0.02            # Fixed interest rate
sigma = 0.30        # Volatility
c = 0.01            # Dividend yield
n = 15

[Since:](http://www.math.nyu.edu/~cai/Courses/Derivatives/lecture8.pdf) *Given interest rate r > 0, it is never optimal to exercise an American call between ex-dividends dates or prior to maturity.*, it is better to exercise the call at maturity. i.e. $t=10$

**1-5,7**

# Homework III

### 1)
*Compute the price of an American call option with strike $K=110$ and maturity $T=.25$ years*

In [2]:
K = 110             # Strike Price
T = 0.25            # Time to maturity

option1 = Binomial_Option(S0, r, sigma, n, T, c)
option1.option_price(K, form="call", style="american")
print("${:.2f}".format(option1.price))

$2.60


### 2)
Compute the price of an American put option with strike $K=110$ and maturity $T=.25$ years.

In [3]:
K = 110             # Strike Price
T = 0.25            # Time to maturity

put1 = Binomial_Option(S0, r, sigma, n, T, c)
put1.option_price(K, form="put", style="american")
print("${:.2f}".format(put1.price))

$12.36


### 3), 4)
*Is it ever optimal to exercise the put of Q2? If so, when it's the earliest?*

It is optimal to exercise an american put option at time $t$, whenever 
$$
\max\{C(t), E[C(t+1)]\} = C(t)
$$

In [4]:
# Substract the intrinsic value of the put (what we would recieve, were we to exercise),
# from the risk-neutral expectation at time t.
# If the intrinsic value is greater than the R.N. expected value, then it would be optimal 
# to early exercise.

compare = np.array(put1.intrinsic_value_tree) - np.array(put1.present_val_tree)[1:]
for t, branch in enumerate(compare[::-1]):
    exercise = [round(x, 3) > 0 for x in branch]
    print("t:{:>2}".format(t), end=" ")
    for v in exercise:
        print("{:>3}".format(v), end=" ")
    print()

t: 0   0 
t: 1   0   0 
t: 2   0   0   0 
t: 3   0   0   0   0 
t: 4   0   0   0   0   0 
t: 5   0   0   0   0   0   1 
t: 6   0   0   0   0   0   1   1 
t: 7   0   0   0   0   0   0   1   1 
t: 8   0   0   0   0   0   0   1   1   1 
t: 9   0   0   0   0   0   0   1   1   1   1 
t:10   0   0   0   0   0   0   0   1   1   1   1 
t:11   0   0   0   0   0   0   0   1   1   1   1   1 
t:12   0   0   0   0   0   0   0   1   1   1   1   1   1 
t:13   0   0   0   0   0   0   0   1   1   1   1   1   1   1 
t:14   0   0   0   0   0   0   0   1   1   1   1   1   1   1   1 


At $t=5$, if the option is *deep in the money*, then it is optimal, and the earliest, to exercise the american put.

### 5)
*Do the call and put option prices of Questions 1 and 2 satisfy put-call parity?*

Recall the put-call parity:
$$
    P_E + S = C_E + K
$$

Since *1)* and *2)* are 'american' options, the put-call parity does **not** hold.

### 6)
Compute the fair value of an American call option with strike $K=110$ and maturity $n=10$ periods where the option is written on a **futures contract** that expires after 15 periods. The futures contract is on the same underlying security of the previous questions.

In [1]:
from FERM import Binomial_Option
putF = Binomial_Option(S0=100, r=0.02, sigma=0.30, n_periods=10, T=0.25, c=0.01, futures=15)
putF.option_price(K=110, form="call", style="american")
print("${:.2f}".format(putF.price))

$2.47


In [13]:
putF.print_tree("asset")

t:  0 100.4
t:  1  95.7 105.2
t:  2  91.2 100.3 110.3
t:  3  87.0  95.7 105.2 115.6
t:  4  82.9  91.2 100.3 110.3 121.2
t:  5  79.1  87.0  95.6 105.1 115.6 127.1
t:  6  75.4  82.9  91.2 100.2 110.2 121.2 133.2
t:  7  71.9  79.0  86.9  95.6 105.1 115.5 127.0 139.7
t:  8  68.5  75.4  82.9  91.1 100.2 110.1 121.1 133.2 146.4
t:  9  65.4  71.9  79.0  86.9  95.5 105.0 115.5 127.0 139.6 153.5
t: 10  62.3  68.5  75.3  82.8  91.1 100.1 110.1 121.0 133.1 146.3 160.9


In [12]:
putF.print_tree("option")

t:  0   2.5
t:  1   1.1   3.8
t:  2   0.4   1.9   5.9
t:  3   0.1   0.7   3.1   8.7
t:  4   0.0   0.2   1.3   5.0  12.5
t:  5   0.0   0.0   0.3   2.3   7.8  17.4
t:  6   0.0   0.0   0.0   0.7   4.0  11.8  23.2
t:  7   0.0   0.0   0.0   0.0   1.3   6.8  17.0  29.7
t:  8   0.0   0.0   0.0   0.0   0.0   2.7  11.1  23.2  36.4
t:  9   0.0   0.0   0.0   0.0   0.0   0.0   5.5  17.0  29.6  43.5
t: 10   0.0   0.0   0.0   0.0   0.0   0.0   0.1  11.0  23.1  36.3  50.9


### 7)
What is the earliest time period in which you might want to exercise the American futures option of Question 6?

In [19]:
compare = np.array(putF.intrinsic_value_tree) - np.array(putF.present_val_tree)[1:]
for t, branch in enumerate(compare[::-1]):
    exercise = [round(x,3) > 0 for x in branch]
    print("t:{:>2}".format(t), end=" ")
    for v in exercise:
        print("{:>3}".format(v), end=" ")
    print()

t: 0   0 
t: 1   0   0 
t: 2   0   0   0 
t: 3   0   0   0   0 
t: 4   0   0   0   0   0 
t: 5   0   0   0   0   0   0 
t: 6   1   0   0   0   0   0   0 
t: 7   1   1   0   0   0   0   0   0 
t: 8   1   1   1   0   0   0   0   0   0 
t: 9   1   1   1   1   0   0   0   0   0   0 


### 8)
*Compute the fair value of a chooser option which expires after n=10 periods. At expiration the owner of the chooser gets to choose (at no cost) a European call option or a European put option. The call and put each have strike K=100 and they mature 5 periods later, i.e. at n=15.*

Since the option is chosen up until $t=10$, we must compute the possible paths of the stock at $t=10$. Once knowing all the possible paths of the stock at that time, we must compute a 5 period option from both the call and put options. For each possible path, take the biggest between the two.  Finally, roll back the binomial tree with the computed prices down to $t=0$

In [22]:
chooser = Binomial_Option(100, r, sigma, 10, T=0.25, c = c)

prices = []

for s10 in chooser.tree[-1]:
    put = Binomial_Option(s10, r, sigma, 5, T=0.25, c=c)
    put.option_price(100, form="put", style="european")
    
    call = Binomial_Option(s10, r, sigma, 5, T=0.25, c=c)
    call.option_price(100, form="call", style="european")
    
    prices.append(np.max([put.price, call.price]))

chooser.tree[-1] = np.array(prices)
chooser.option_price(K=0)

print("${:.2f}".format(chooser.price))

$14.28


In [25]:
chooser.print_tree("option")

t:  0  14.3
t:  1  13.9  14.7
t:  2  14.5  13.2  16.3
t:  3  16.1  12.8  13.7  19.1
t:  4  18.5  13.6  12.1  15.3  23.0
t:  5  21.4  15.4  11.7  12.5  18.3  27.9
t:  6  24.7  18.1  12.6  10.7  14.3  22.5  33.6
t:  7  28.0  21.3  14.8  10.4  11.1  17.6  27.6  39.7
t:  8  31.2  24.6  17.8  11.6   9.1  13.2  22.1  33.4  46.4
t:  9  34.4  28.0  21.1  14.4   8.8   9.4  17.1  27.4  39.6  53.4
t: 10  37.4  31.2  24.6  17.6  11.1   6.4  12.6  21.8  33.3  46.3  60.8
